# ERP 원본 데이터
- 제품군-GITEM-SITEM: GITEM 정보
- GITEM_ 공정-순서 (공정군 분류 정보 추가)
- 수율(공정사용 기준 미정)
- 라인스피드 (확정)
- P/O 정보: 기준 X, 현재로서는 이전에 있던 정보 그대로 사용

# 생산계획 원본 데이ㅌ터
- 폭변경: 수정없이 유지
- 공정교체시간: 호기코드로 변경 및 호기코드 컬럼명 수정

# 사용자 입력
- 기준날짜 (base_date)
- 윈도우 크기 (window_size)
- 사용하는 수율/라인스피드 데이터 기준 (yield_period, linespeed_period)

# ==============================================

In [1]:
import pandas as pd

# 사용 안 함
gitem_info = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "제품군-GITEM-SITEM", skiprows = 2)
gitem_info = gitem_info.drop(columns = {"영문명"})

# PYTHON_INPUT: 1. ORDER_DATA
order = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "PO정보", skiprows = 1)

order_data = order[['PoNo','GItemNo','GItemName', 'sitemno', 'SitemName', 'Spec', 'IpgmQty','DUEDATE']]
order_data[['Thickness', 'Width', 'Length']] = order_data['Spec'].str.split("*", expand=True)

order_data['Fabric_Length'] = order_data['Length'].astype(int) * order_data['IpgmQty'].astype(int)

order_data = order_data.rename(columns = {"GItemNo":"GitemNo", "GItemName":"GitemName"})

# PYTHON_INPUT: 2. LINESPEED
linespeed = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "라인스피드-GITEM등", skiprows = 5)
linespeed = linespeed.drop(columns = {"영문명"})
linespeed = linespeed.drop_duplicates(keep='first') # 중복된 행은 첫 행만 남기고 삭제


import re

def linespeed_selection(linespeed, linespeed_period):

    if linespeed_period == '6_months':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*1$", col)] # ㅣ로 시작, 1로 끝나는 컬럼
        print(f"6개월 기준 컬럼명{linespeed_cols}") 
    elif linespeed_period == '1_year':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*2$", col)] # L로 시작, 2로 끝나는 컬럼
        print(f"1년 기준 컬럼명{linespeed_cols}") 
    elif linespeed_cols == '3_months':
        linespeed_cols = [col for col in linespeed.columns if re.match(r"^L.*2$", col)] # L로 시작, 2로 끝나는 컬럼
        print(f"3개월 기준 컬럼명{linespeed_cols}") 
    # 첫번째로 NaN이 아닌 값으로 Linespeed 채우기
    linespeed['linespeed'] = linespeed[linespeed_cols].bfill(axis=1).iloc[:, 0] 

    # linespeed로 선택된 컬럼명 가져오기
    linespeed['selected'] = linespeed[linespeed_cols].apply(lambda x: x.first_valid_index(), axis=1)
    
    # 사용하지 않는 컬럼(L로 시작하는 컬럼) 제외
    linespeed = linespeed.drop(columns = linespeed.columns[linespeed.columns.str.startswith("L")])
    return linespeed
    
    
selected_linespeed = linespeed_selection(linespeed, linespeed_period = '6_months')

linespeed_pivot = selected_linespeed.pivot(
    index = ['GitemNo','PROCCODE'],
    columns = 'MachineNo',
    values = 'linespeed'
).reset_index().rename_axis(None, axis=1)



,DeptGbn,DeptGbnNAme,Grp1,GRP1_NAME,Grp2,GRP2_Name,Grp3,GRP3_Name,GItemNo,GItemName,SItemNo,CodeName,Thick,IWidth,ILength,Spec
0,2,윈도우 제품군,3,HIGH-END,1,C-PPF,1,C-PPF,31798,NaN,NaN,NaN,NaN,1524,31,10*1524*31


In [4]:
# PYTHON_INPUT: 3. OPERATION_TYPES
gitem_operation_sequence = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "GITEM-공정-순서", skiprows = 1)
gitem_operation_sequence = gitem_operation_sequence.drop(columns = {"영문명"})


operation_types = gitem_operation_sequence[['PROCCODE', 'PROCNAME','ProcGbn']].drop_duplicates(keep = 'first')
operation_types

,PROCCODE,PROCNAME,ProcGbn
0,20300,염료점착,염료
1,20700,유광 S/R,SR
2,12400,TOP COATING,SR
3,23311,PPF점착,투명
4,20500,투명점착,투명
5,20706,MIBK SR,SR
6,20902,안료접착,안료
7,20906,인쇄,인쇄


In [5]:
# PYTHON_INPUT: 4. OPERATION_SEQUENCE
gitem_operation_sequence = gitem_operation_sequence.rename(columns = {"GITEMNO":"GitemNo", "GItemName":"GitemName"})
gitem_operation_sequence

,GitemNo,GitemName,PROCCODE,PROCNAME,PROCSEQ,ProcGbn
0,31540,NaN,20300,염료점착,1,염료
1,31540,NaN,20700,유광 S/R,2,SR
2,31704,NaN,12400,TOP COATING,2,SR
3,31704,NaN,23311,PPF점착,1,투명
4,31798,NaN,20500,투명점착,3,투명
5,31798,NaN,20706,MIBK SR,2,SR
6,31798,NaN,20902,안료접착,1,안료
7,31798,NaN,20906,인쇄,4,인쇄
8,31800,NaN,20500,투명점착,3,투명
9,31800,NaN,20706,MIBK SR,2,SR


In [6]:
# PYTHON_INPUT: 5. YIELD

yield_data = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "수율-GITEM등", skiprows = 5)
yield_data = yield_data.rename(columns = {"GItemName":"GitemName"})
yield_data = yield_data.drop(columns = {"영문명"})


import re
import numpy as np

def yield_selection(yield_data, yield_period):

    if yield_period == '6_months':
        yield_cols = [col for col in yield_data.columns if re.match(r"^S.*1$", col)] # S로 시작, 1로 끝나는 컬럼
        print(f"6개월 기준 컬럼명{yield_cols}") 
    elif yield_period == '1_year':
        yield_cols = [col for col in yield_data.columns if re.match(r"^S.*2$", col)] # S로 시작, 2로 끝나는 컬럼
        print(f"1년 기준 컬럼명{yield_cols}") 
    elif yield_period == '3_months':
        yield_cols = [col for col in yield_data.columns if re.match(r"^S.*2$", col)] # S로 시작, 2로 끝나는 컬럼
        print(f"3개월 기준 컬럼명{yield_cols}") 
        
    # 값이 0이면 nan으yield_data = 로 변경 
    yield_data = yield_data.replace(0, np.nan)
    # 첫번째로 NaN이 아닌 값으로 Linespeed 채우기
    yield_data['yield'] = yield_data[yield_cols].bfill(axis=1).iloc[:, 0] 
    
    # 수율 없으면 100으로 변경
    yield_data['yield'] = yield_data['yield'].replace(np.nan, 100)

    # linespeed로 선택된 컬럼명 가져오기
    yield_data['selected'] = yield_data[yield_cols].apply(lambda x: x.first_valid_index(), axis=1)
    
    # 사용하지 않는 컬럼(L로 시작하는 컬럼) 제외
    yield_data = yield_data.drop(columns = yield_data.columns[yield_data.columns.str.startswith("S")])
    return yield_data
    
    
yield_info = yield_selection(yield_data, yield_period = '6_months')
yield_info = yield_info.drop_duplicates(subset = ['GitemNo', 'GitemName'], keep = 'first')
yield_info

6개월 기준 컬럼명['S11', 'S21', 'S31', 'S41', 'S51']


,GitemNo,GitemName,yield,selected
0,25003,SCF-NS-BS03,99.01,S11
2,25008,SCF-NS-BS52,98.41,S11
4,25026,SCF-NS-SS03,98.89,S11
6,25029,SCF-NS-SS19,98.80,S11
8,25030,SCF-NS-SS35,98.90,S11
...,...,...,...,...
1606,32573,SLCF-XQ420N,100.00,None
1611,32599,RAINBOW(KOR)-ND-Legend Grace 3397,100.00,None
1612,32353,LGD Privacy Lenticular Pattern,100.00,None
1624,32444,QBF-2-130-B3(A),100.00,None


In [7]:
# PYTHON_INPUT: 6. MACHINE_MASTER_INFO
machine_master_info = (
    linespeed[['MachineNo', 'MachineName']]
    .drop_duplicates()
    .sort_values(by = 'MachineNo')
    .reset_index(drop=True)
    .assign(MachineIndex=lambda df: range(len(df)))
    )
machine_master_info

,MachineNo,MachineName,MachineIndex
0,A2020,AgNW2호기,0
1,C2010,코팅1호기_WIN,1
2,C2250,코팅25호기_WIN,2
3,C2260,코팅26호기_WIN,3
4,C2270,코팅27호기_WIN,4
5,D2280,코팅28호기_DSP,5
6,O2310,코팅31호기_DSP,6
7,O2320,코팅32호기_DSP,7
8,O2340,코팅34호기_DSP,8
9,O2360,코팅36호기_DSP,9


In [8]:
# PYTHON_INPUT: MIXTURE
mixture_data = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "배합액정보", skiprows = 5)
mixture_data = mixture_data.drop(columns = {"영문명"})

mixture_data = mixture_data[['GitemNo', 'PROCCODE',  'Che1', 'Che2']]
mixture_data

,GitemNo,PROCCODE,Che1,Che2
0,25003,20300,T02629,NaN
1,25008,20300,T02582,NaN
2,25026,20300,W2017,NaN
3,25026,20700,T02149,T0873
4,25029,20300,W2018,NaN
...,...,...,...,...
954,32762,20902,T02616,NaN
955,32763,20601,W2085,NaN
956,32763,23100,T0897,NaN
957,32764,20601,T02601,W2086


In [9]:
# PYTHON_INPUT: 7. OPERATION_DELAY
operation_delay = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "공정교체시간", skiprows = 1)
operation_delay.head(1)

,prev_operation_type,next_operation_type,type_change_time
0,SR,SR,2


In [10]:
# PYTHON_INPUT: 8. WIDTH_CHANGE
width_change = pd.read_excel("생산계획 필요기준정보 내역-Ver4.xlsx", sheet_name = "폭변경", skiprows = 1)
width_change

,MachineNo,long_to_short,short_to_long
0,C2010,2,2
1,C2260,4,4
2,C2270,2,2


In [11]:
machine_limit = pd.DataFrame(columns=['PROCCODE', 'MachineNo']) 
machine_allocate = pd.DataFrame(columns=['PROCCODE', 'MachineNo']) 
machine_rest = pd.DataFrame(columns=['MachineNo', 'dt_start', 'dt_end'])

In [12]:
with pd.ExcelWriter("python_input.xlsx", engine="openpyxl") as writer:
    linespeed_pivot.to_excel(writer, sheet_name="linespeed", index=False) # 변경사항: 공정명 -> 공정코드
    operation_types.to_excel(writer, sheet_name="operation_types", index=False) # 변경사항 X
    # gitem_operation 필요 X
    gitem_operation_sequence.to_excel(writer, sheet_name="operation_sequence", index=False) # 변경사항: operation_types 관련 정보 추가, 배합액 정보 삭제
    yield_info.to_excel(writer, sheet_name="yield_data", index=False)
    machine_master_info.to_excel(writer, sheet_name = "machine_master_info", index=False) # 변경사항 X
    mixture_data.to_excel(writer, sheet_name = "mixture_data", index=False) # 추가된 시트. 이전에는 operation_sequence에 종속되어있었음
    operation_delay.to_excel(writer, sheet_name = "operation_delay", index=False) 
    width_change.to_excel(writer, sheet_name = "width_change", index=False) # 호기코드로 변경
    machine_limit.to_excel(writer, sheet_name = "machine_limit", index=False) # 호기코드, 공정코드로 변경 
    machine_allocate.to_excel(writer, sheet_name = "machine_allocate", index=False) # 호기코드, 공정코드로 변경
    machine_rest.to_excel(writer, sheet_name = "machine_rest", index=False) # 호기코드로 변경
    order_data.to_excel(writer, sheet_name = "order_data", index=False) # 호기코드로 변경
    